In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../SentimentAnalysis/GPT/secret.env")

api_key = os.getenv("GEMINI_API_KEY")

if api_key is None:
    print("Error: GEMINI_API_KEY not found in .env file or environment variables.")
else:
    print("GEMINI_API_KEY loaded successfully.")

GEMINI_API_KEY loaded successfully.


In [2]:
import google.generativeai as genai
from tqdm import tqdm

genai.configure(api_key=api_key)
generation_config = genai.GenerationConfig(
        temperature=0,
)
model = genai.GenerativeModel("gemini-2.0-flash-001", generation_config=generation_config)

In [3]:
from google.api_core import retry

# Catch transient Gemini errors.
def is_retryable(e) -> bool:
    if retry.if_transient_error(e):
        # Good practice, but probably won't fire with the google-genai SDK
        return True
    elif (isinstance(e, genai.errors.ClientError) and e.code == 429):
        # Catch 429 quota exceeded errors
        return True
    elif (isinstance(e, genai.errors.ServerError) and e.code == 503):
        # Catch 503 model overloaded errors
        return True
    else:
        return False

@retry.Retry(predicate=is_retryable)
def generate_content_with_rate_limit(prompt):
  return model.generate_content(prompt).text

In [4]:
prompt = "Translate the following English sentence to Thai: [TEXT] (Do not provide any explanation, just the translation.)"

In [7]:
import pandas as pd
import numpy as np

filename = "Gemini_news_2025-05-28_01-11-58.csv"
df = pd.read_csv(f"../SentimentAnalysis/GPT/Output/Gemini/{filename}")

In [8]:
translate = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Prompting"): 

    filled_prompt = prompt.replace("[TEXT]", row["summary"])

    try:
        response = generate_content_with_rate_limit(filled_prompt)
        finalprediction = response.strip()
        if not finalprediction:
            print(f"Row {index}: LLM returned an empty string.")
            translate.append("LLM_EMPTY_RESPONSE")
        else:
            translate.append(finalprediction)
    except ValueError as ve:
        print(f"Row {index}: ValueError - {ve}. Appending 'ERROR_VALUE_ERROR'.")
        translate.append("ERROR_VALUE_ERROR")
        continue
    except Exception as e:
        if "429 Too Many Requests" in str(e) or "quota" in str(e).lower():
            print(f"Row {index}: Rate Limit Exceeded or Quota Error - {e}. Appending 'ERROR_RATE_LIMIT'.")
            translate.append("ERROR_RATE_LIMIT")
        elif "safety" in str(e).lower() or "blocked" in str(e).lower():
             print(f"Row {index}: Content Safety/Blocked - {e}. Appending 'ERROR_SAFETY_BLOCKED'.")
             translate.append("ERROR_SAFETY_BLOCKED")
        else:
            print(f"Row {index}: Unexpected Error - {e}. Appending 'ERROR_UNEXPECTED'.")
            translate.append("ERROR_UNEXPECTED")
        continue

Prompting: 100%|██████████| 20/20 [00:58<00:00,  2.91s/it]


In [9]:
df["translate"] = translate
df.to_csv(f"ForVisualize/{filename}", index=False)